# Gramática Libre de Contexto
## Lectura de archivos

In [413]:
file = open(r"C:\Users\hrosa\OneDrive\Documentos\CIC IPN\Primer Semestre\Teoria Computacion\GLC\GLC.txt")

In [414]:
content = file.read()

In [415]:
content.split('\n')

['S -> AB', 'A -> E', 'B -> C', 'C -> D', 'D -> b', 'E -> a']

In [416]:
split_content = content.split('\n')
stack = []
for r in split_content:
    rules = []
    for s in r.split():
        rules.append([s])
    stack.append(rules)
print(f'stack: {stack}')

stack: [[['S'], ['->'], ['AB']], [['A'], ['->'], ['E']], [['B'], ['->'], ['C']], [['C'], ['->'], ['D']], [['D'], ['->'], ['b']], [['E'], ['->'], ['a']]]


In [417]:
#se crean los diccionarios de cada regla de inferencia
rules = {}
for i in stack:
    if not i:
        continue
    head = i[0][0]
    productions = []
    for j in i[2:]:
        if j[0] != '|':
            productions.append(j[0])
            print(productions)
    rules[head] = productions
print(f'rules{rules}')

['AB']
['E']
['C']
['D']
['b']
['a']
rules{'S': ['AB'], 'A': ['E'], 'B': ['C'], 'C': ['D'], 'D': ['b'], 'E': ['a']}


## Simplificación de unificación no recursiva

In [390]:
def unificacion_no_recursiva(rules):
    #simplificacion de unificacion no recursiva
    for k,v in rules.items():
        # print(f'k:{k}')
        # print(f'v:{v}')
        first = {}
        for i in v: #por cada elemento de producciones
            max_prefix = i #como no se tiene compun, se toma todo el elemento
            # print(f'HOLA I{i}')
            for j in v: #por cada aparición de elemento: comparar contra los demás
                if j != i: #si esa comparación no es igual
                    # print(f'j:{j} != i:{i}')
                    prefix_common = "" #se inicializa en cero, porque aún no hay prefijo comúb
                    compare = 1 #caracter en común en comparación 
                    #mientras no tenga longitud mayor y haya comun
                    #compare 1: a == a
                    #compare 2: ab != B
                    while compare <= len(i) and compare <= len(j) and i[:compare] == j[:compare]:
                        prefix_common = i[:compare]
                        compare += 1

                    counter = 0
                    for production in v: #cuenta # de elementos con prefijo compun
                        if production.startswith(prefix_common):
                            counter += 1
                    if prefix_common and counter > 1: #actualizar el término más común
                        max_prefix = prefix_common

            prefix_found = max_prefix
            body = i[len(prefix_found):] #no incluir el prefijo común

            if prefix_found not in first:
                first[prefix_found] = []
            first[prefix_found].append(body)
    #     print(first)

    # print(f'new rules: {first}')
    new_rules = {}
    counter_r = 1
    # print(rules)
    origin_rules = None
    for k, v in rules.items():
        origin_rules = k

    productions_origin = []

    for k,v in first.items():
        if len(v) == 1: #si solo hay una producción:
            new_rules[k] = v
            productions_origin.append(k+v[0])
        else:
            aux = f"R{counter_r}"
            counter_r += 1
            new_rules[aux] = v
            productions_origin.append(k+aux)

    rules_unified = {origin_rules:productions_origin}
    for k,v in new_rules.items():
        rules_unified[k] = v

    # print(f'new rules: {rules_unified}')

    for k,v in rules_unified.items():
        print(f'{k}->{"|".join(v)}')

    return rules_unified

In [392]:
unificacion_no_recursiva(rules)

A->aR1|cdR2
R1->bB|B
R2->g|eB|fB


{'A': ['aR1', 'cdR2'], 'R1': ['bB', 'B'], 'R2': ['g', 'eB', 'fB']}

## Simplicidad de símbolos redundantes
1. variables no productivas.
2. variables inalcanzables.

In [411]:
def simbolos_redundantes(rules):
    not_productive = []
    for k,v in rules.items():
        # print(f'k:{k} and v:{v}')
        productive = False

        for production in v:
            terminal_count = 0
            no_terminal_count = False

            for char in production:
                if char.islower(): #es terminal?
                    terminal_count += 1
                if char.isupper():
                    no_terminal_count = True

            if terminal_count >= 1 and not no_terminal_count:
                productive = True
                
        if not productive and k != 'S':
            not_productive.append(k)
    # print(not_productive)

    #=======
    filter_no_productive = {}
    for k,v in rules.items():
        if k not in not_productive:
            new_rules = []
            for production in v:
                productive = True
                for char in production:
                    if char in not_productive:
                        # print(char)
                        productive = False
                if productive:
                    new_rules.append(production)
            if new_rules:
                filter_no_productive[k] = new_rules
    # print(filter_no_productive)

    #====
    rule_s = filter_no_productive['S']
    # print(f'rules s: {rule_s}')
    not_redundants = {}
    inalcanzable = None
    for k, v in filter_no_productive.items():
        for production in v:
            # print(production)
            # print(f'k:{k}')
            for i in rule_s:          
                # print(i)
                # print(f'---{k}')
                if k not in i and k != 'S':
                    inalcanzable = k
    # print(inalcanzable)

    for k,v in filter_no_productive.items():
        if k != inalcanzable:
            not_redundants[k] = v
    # print(not_redundants)

    return not_redundants

In [412]:
simbolos_redundantes(rules)

{'S': ['bX'], 'X': ['ad']}

## Simplicidad por reglas unitarias

In [419]:
def simplicidad_unitaria(rules):
    new_rules_unity = {}
    new_rules_unity['S'] = rules['S']
    symbols_s = []
    for k,v in new_rules_unity.items():
        for production in v:
            for char in production:
                symbols_s.append(char) #se añaden los terminales en symbols_s

    # print(symbols_s)

    for i in symbols_s:
        if i.isupper():
            current_no_terminal = i
            consumido = []
            #no terminal está en reglas?, es un no terminal?, es un no terminal?
            while current_no_terminal in rules and len(rules[current_no_terminal]) == 1 and rules[current_no_terminal][0].isupper():
                if current_no_terminal in consumido: #está ese no terminal en consumido?
                    break
                #no está, agrégalo a consumido
                consumido.append(current_no_terminal)
                # print(f'consumido:{consumido}')
                current_no_terminal = rules[current_no_terminal][0] #valor de llave actual
                # print(current_no_terminal)
            #una vez que hay un terminal...
            if current_no_terminal in rules: #no terminal en rules?
                new_rules_unity[i] = [rules[current_no_terminal][0]] #entonces i (A, por ejemplo) es rules['D']
            else:
                # print('yes')
                new_rules_unity[i] = current_no_terminal[0]
    # print(f'new rules:{new_rules_unity}')
    return new_rules_unity

In [420]:
simplicidad_unitaria(rules)

{'S': ['AB'], 'A': ['a'], 'B': ['b']}

## Simplicidad por epsilon-producciones

In [321]:
epsilon = [] #todos los elementos que pueden llevar a epsilon

for k, v in rules.items():
    if 'epsilon' in v and k not in epsilon: #una de las producciones contiene epsilon?
        epsilon.append(k)
# print(epsilon)
updates = True

while updates: #mientras exista un epsilon...
    updates = False

    for k, v in rules.items():
        if k in epsilon: #si key está, avanzamos
            continue
        for production in v: #comparar producción contra todas
            condition_epsilon = True #puede haber epsilon

            for char in production: #por cada caracter de producción
                # print(f'char: {char}')
                if char not in epsilon: #si caracter no está en epsilon; a no en [X]
                    condition_epsilon = False #no existe epsilon
                    break

            if condition_epsilon: #si hay.. se añade a epsilon
                # print('yes')
                epsilon.append(k)
                updates = True
                break

new_rules_epsilon = {}

for k,v in rules.items():
    new_rules_epsilon[k] = []
    for production in v:
        if production == 'epsilon': #eliminar epsilon
            # print('oh',production)
            continue

        if production not in new_rules_epsilon[k]:
            new_rules_epsilon[k].append(production)
        # print(new_rules_epsilon)

        pos_epsilon = []

        for i in range(len(production)):
            if production[i] in epsilon:
                pos_epsilon.append(i)

        combinations = [[]]

        for pos in pos_epsilon:
            new_combinations = []
            for c in combinations:
                new_combinations.append(c)
                new_combinations.append(c + [pos])

            combinations = new_combinations

        for comb in combinations:
            if not comb:
                continue
            new_production = ""

            for i in range(len(production)):
                if i not in comb:
                    new_production += production[i]
            if new_production != "" and new_production not in new_rules_epsilon[k]:
                new_rules_epsilon[k].append(new_production)
                
print(new_rules_epsilon)


{'S': ['ABaC', 'ABa', 'AaC', 'Aa', 'BaC', 'Ba', 'aC', 'a'], 'A': ['BC', 'B', 'C'], 'B': ['b'], 'C': ['D'], 'D': ['d']}


## Forma Normal de Chomsky (FNC)